In [59]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.pipeline import Pipeline
import os
from sklearn.ensemble import GradientBoostingClassifier,RandomForestRegressor
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from pandas import ExcelWriter
from sklearn.ensemble import RandomForestClassifier
import openpyxl
from sklearn import metrics
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [60]:
import warnings
warnings.filterwarnings('ignore')

In [61]:
outliers=[]
def detect_outlier(data_1):
    
    threshold=1.5
    mean_1 = np.mean(data_1)
    std_1 =np.std(data_1)
    
    
    for y in data_1:
        z_score= (y - mean_1)/std_1 
        if  z_score > threshold:
            outliers.append(y)
    return outliers

In [62]:
def process_data(data_train,data_val,cv): 
    report_All= []
    x_column_list = data_train.drop(columns=['y_c']).columns  
    #Classification RF  
    
    pipeRF = Pipeline([('classifier', [RandomForestRegressor()])])  
    param_grid = [  
    {'classifier' : [RandomForestRegressor()], 
    'classifier__n_estimators': [100, 200],
    'classifier__min_samples_split': [8, 10],
    'classifier__min_samples_leaf': [3, 4, 5],
    'classifier__max_depth': [80, 90]}]  

    
    clf = GridSearchCV(pipeRF, param_grid = param_grid, cv = cv, n_jobs=-1, scoring='neg_mean_squared_error')  
    # Fit on data  
    clf.fit(data_train[x_column_list],data_train['y_c'])  
    best_clf=clf.best_estimator_  
    y_valid=best_clf.predict(data_val[x_column_list])
    
    report_All.append(metrics.mean_absolute_error(data_val['y_c'].values, y_valid))
    report_All.append(metrics.mean_squared_error(data_val['y_c'].values, y_valid))
    report_All.append(metrics.mean_squared_error(data_val['y_c'].values, y_valid))
    report_All.append(r2_score(data_val['y_c'].values, y_valid))
    #report_All = classification_report(data_val['y_c'],y_valid,output_dict=True) 
    
    dAll=pd.DataFrame(report_All)
    return dAll

In [63]:
cv=RepeatedKFold(n_splits=10,n_repeats=3, random_state=100)

In [ ]:
path_response='/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/python code local/Main Data Files/yield_response/'  
for file_response in os.listdir(path_response):  
    if (file_response != '.DS_Store') & (file_response != 'Icon\r'):  
        print(file_response)  
        path_r= path_response+file_response  
        os.chdir(path_r)  
        for re in os.listdir(path_r):  
            if re[0:8] == 'response':  
                response = pd.read_csv(path_response+file_response+'/'+re)  
                response.rename(columns={'Column1':'Link_ID',response.columns[1]:'y_c'}, inplace=True)  
                #response.drop(columns=response.columns[2], inplace=True)  
                #response=response.drop(columns='Variety2')  
                path_x = '/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/python code local/Main Data Files/normalized_data_sklearn/'  
                writer= pd.ExcelWriter(path_r+'/'+'classification_RF'+'.xlsx', engine='xlsxwriter')   
                for file_folder in os.listdir(path_x):  
                    if (file_folder[-4:] != '.csv') & (file_folder != '.DS_Store')& (file_folder != 'Icon\r'):          
                        path = path_x+file_folder  
                        os.chdir(path)  
                        file_list = []  
                        tRF=pd.DataFrame()  
                        tcluster=pd.DataFrame()  
                        k=0  
                        for file in os.listdir(path):  
                            if (file[0] != 't') & (file[-4:] == '.csv') & (file != '.DS_Store')& (file_folder != 'Icon\r'):  
                                print(file)  
                                file_list.append(file)  
                                data_temp = pd.read_csv(file)  
                                data_temp.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True)  
                                data=pd.merge(response,data_temp,on='Link_ID')  
                                data.drop(columns = 'Link_ID',inplace=True) 
                                
                                OT = pd.DataFrame(detect_outlier(data['y_c']))
                                data['y_c'] = np.where(data['y_c'] >OT.min()[0], OT.min()[0],data['y_c'])
                                
                                data_train,data_val = train_test_split(data,train_size=0.8, random_state=42)  
                                output = process_data(data_train,data_val,cv)  
                                tRF[k]=output        
                                k=k+1  
                        tRF.to_excel(writer, sheet_name=file_folder, index=True)  
                writer.save()                      

yield_per_meter
 1 _ 3 .csv
 4 _ 1 .csv
 5 _ 1 .csv
 1 _ 2 .csv
 1 _ 4 .csv
 3 _ 1 .csv
 2 _ 1 .csv
 2 _ 2 .csv
 3 _ 2 .csv
 4 _ 4 .csv
 5 _ 4 .csv
 2 _ 3 .csv
 3 _ 3 .csv
 1 _ 1 .csv
 3 _ 4 .csv
 2 _ 4 .csv
 5 _ 3 .csv
 4 _ 3 .csv
 5 _ 2 .csv
 4 _ 2 .csv
 1 _ 3 .csv
 4 _ 1 .csv
 5 _ 1 .csv
 1 _ 2 .csv
 1 _ 4 .csv
 3 _ 1 .csv
 2 _ 1 .csv
 2 _ 2 .csv
 3 _ 2 .csv
 4 _ 4 .csv
 5 _ 4 .csv
 2 _ 3 .csv
 3 _ 3 .csv
 1 _ 1 .csv
 3 _ 4 .csv
 2 _ 4 .csv
 5 _ 3 .csv
 4 _ 3 .csv
 5 _ 2 .csv
 4 _ 2 .csv
 1 _ 3 .csv
 4 _ 1 .csv
 5 _ 1 .csv
 1 _ 2 .csv
 1 _ 4 .csv
 3 _ 1 .csv
 2 _ 1 .csv
 2 _ 2 .csv
 3 _ 2 .csv
 4 _ 4 .csv
 5 _ 4 .csv
 2 _ 3 .csv
 3 _ 3 .csv
 1 _ 1 .csv
 3 _ 4 .csv
 2 _ 4 .csv
 5 _ 3 .csv
 4 _ 3 .csv
 5 _ 2 .csv
 4 _ 2 .csv
 1 _ 3 .csv
 4 _ 1 .csv
 5 _ 1 .csv
 1 _ 2 .csv
 1 _ 4 .csv
 3 _ 1 .csv
 2 _ 1 .csv
 2 _ 2 .csv
 3 _ 2 .csv
 4 _ 4 .csv
 5 _ 4 .csv
 2 _ 3 .csv
 3 _ 3 .csv
 1 _ 1 .csv
 3 _ 4 .csv
 2 _ 4 .csv
 5 _ 3 .csv
 4 _ 3 .csv
 5 _ 2 .csv
 4 _ 2 .csv
 1 _ 3 .csv
 4 _ 1 .csv


# Feature Selection

In [10]:
data_path2 = '/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/python code local/Main Data Files/feature_ML_disease'

In [14]:
path_list2 = []
#reading files in folder response
for root, dirs, files in os.walk(data_path2, topdown=False):
    for path in dirs:
        path_list2.append(path)
#reading sheet name       
wb = openpyxl.load_workbook(data_path2+'/'+path+'/feature_selection.xlsx')
sheet_list = wb.sheetnames

results_dic = dict.fromkeys(sheet_list)

for sheet_name in results_dic.keys():
    temp_df = pd.DataFrame(columns=path_list2, index=range(0,700))
    for folder in path_list2:
        data_temp = pd.read_excel(data_path2+'/'+folder+'/feature_selection.xlsx', sheet_name=sheet_name)
        temp_df[folder].iloc[
            range(0, len(data_temp['Unnamed: 0'].values))] = data_temp['Unnamed: 0'].values

    results_dic[sheet_name] = temp_df
    
for key in results_dic.keys():
    results_dic[key] = results_dic[key].iloc[range(0,max(30, round(len(data_temp)/3)))]

In [15]:
data_path = '/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/python code local/Main Data Files/feature_netcomi_disease'

path_list = []

for root, dirs, files in os.walk(data_path, topdown=False):
    for path in dirs:
        path_list.append(path)

In [16]:
list_level = ['Class', 'Family', 'Genus', 'Order', 'Phylum']

all_data = dict.fromkeys(dirs)

for folder in path_list:
    
    all_data[folder] = dict.fromkeys(list_level)
    temp_path = data_path+'/'+folder
    temp_file_list = os.listdir(temp_path)
    for file in temp_file_list:
        if 'Count' in file:
            level = file.split('_')[-1][:-4]
            data_temp = pd.read_csv(data_path+'/'+folder+'/'+file,index_col=0)
            data_temp.sort_values(by='dif_close', ascending=False, inplace=True)
            #all_data[folder][level] = data_temp.iloc[range(0,max(40, round(len(data_temp)/3)))]
            all_data[folder][level] = data_temp.iloc[range(0,max(30, round(len(data_temp)/3)))]
            #print(level,":",max(30 ,round(len(data_temp)/3)))

print('Done.')

Done.


In [17]:
list_level = ['Class', 'Family', 'Genus', 'Order', 'Phylum']
SELECTED_FEATURE = dict.fromkeys(dirs)
for col in range(0,len(list_level)):
    level = list_level[col]
    print(level)
    SELECTED_FEATURE[level] = dict.fromkeys(dirs)
    response_list = path_list
    feature_list = []
    for response in response_list:
        feature_list.append(all_data[response][level].index.union(results_dic[level][response][results_dic[level][response].notnull()].values))
    feature_list = [item for subitem in feature_list for item in subitem]
    feature_list = np.unique(feature_list)
    print(len(feature_list))
    matrix_df = pd.DataFrame(columns = response_list, index = feature_list)
    for response in response_list:
        for feature in feature_list:
                if (feature in all_data[response][level].index) & (feature in results_dic[level][response].values):
                    matrix_df[response].loc[feature] = 3
                elif(feature in all_data[response][level].index) &(feature not in results_dic[level][response].values):
                    matrix_df[response].loc[feature] = 2##NetComi
                elif (feature not in all_data[response][level].index) &(feature in results_dic[level][response].values):
                    matrix_df[response].loc[feature] = 1##ML
                else:
                    matrix_df[response].loc[feature] = 0##NotMLnotNetcomi
            
        #print(matrix_df[response])
#         ree=re[matrix_df[response]==3]
#         print(ree)
#         SELECTED_FEATURE[response][level]= ree.index     
        SELECTED_FEATURE[level][response] = matrix_df[response][matrix_df[response]==3].index
    print(SELECTED_FEATURE[level][response].shape[0])
    matrix_df['Sum'] = 0
    for index in matrix_df.index:
        matrix_df['Sum'].loc[index] = sum(matrix_df.loc[index].values)
    
    matrix_df.sort_values(by='Sum', ascending=False, inplace=True)
    matrix_df=matrix_df.iloc[1:30,:]
  

Class
76
7
Family
144
8
Genus
247
12
Order
133
10
Phylum
42
21


In [ ]:
for file_response in os.listdir(path_response):  
    if (file_response != '.DS_Store') & (file_response != 'Icon\r'):  
        print(file_response)  
        path_r= path_response+file_response  
        os.chdir(path_r)  
        for re in os.listdir(path_r):  
            if re[0:8] == 'response':  
                response = pd.read_csv(path_response+file_response+'/'+re)  
                response.rename(columns={'Column1':'Link_ID','x1':'y_b'}, inplace=True)  
                #response.drop(columns=response.columns[2], inplace=True)  
                #response=response.drop(columns='Variety2')  
                path_x = '/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/python code local/Main Data Files/normalized_data_sklearn/'  
                writer= pd.ExcelWriter(path_r+'/'+'classification_RF_FS'+'.xlsx', engine='xlsxwriter')   
                for file_folder in os.listdir(path_x):  
                    if (file_folder[-4:] != '.csv') & (file_folder != '.DS_Store')& (file_folder != 'Icon\r'):          
                        path = path_x+file_folder  
                        os.chdir(path)  
                        file_list = []  
                        tRF=pd.DataFrame()  
                        tcluster=pd.DataFrame()  
                        k=0  
                        for file in os.listdir(path):  
                            if (file[0] != 't') & (file[-4:] == '.csv') & (file != '.DS_Store')& (file_folder != 'Icon\r'):  
                                print(file)  
                                file_list.append(file)  
                                data_temp = pd.read_csv(file) 
                                data_temp.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True) 
                                A = [item for item in SELECTED_FEATURE[file_folder][file_response]]
                                A.insert(0, "Link_ID")
                                data_temp = data_temp[A]
                                data_temp.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True)  
                                data=pd.merge(response,data_temp,on='Link_ID')
                                
                                data.drop(columns = 'Link_ID',inplace=True)  
                                data_train,data_val = train_test_split(data,train_size=0.8, random_state=42)  
                                output = process_data(data_train,data_val,cv)  
                                tRF[k]=pd.DataFrame(output['f1-score'].values)         
                                k=k+1  
                        tRF.to_excel(writer, sheet_name=file_folder, index=True)  
                writer.save()                      

no_tuber_scab
 1 _ 3 .csv


In [20]:
 SELECTED_FEATURE

{'no_tuber_scab': None,
 'no_tuber_scabpit': None,
 'no_tuber_scabsuper': None,
 'Class': {'no_tuber_scab': Index(['Gitt.GS.136', 'Holophagae', 'Ktedonobacteria', 'Microgenomatia',
         'Negativicutes', 'Nitrospiria', 'OLB14', 'Parcubacteria',
         'S0134.terrestrial.group', 'Saccharimonadia', 'Subgroup.22'],
        dtype='object'),
  'no_tuber_scabpit': Index(['Gemmatimonadetes', 'Gitt.GS.136', 'Holophagae', 'JG30.KF.CM66',
         'Ktedonobacteria', 'OLB14', 'Oligoflexia', 'Planctomycetes',
         'Polyangia', 'Saccharimonadia'],
        dtype='object'),
  'no_tuber_scabsuper': Index(['Gammaproteobacteria', 'Ktedonobacteria', 'Myxococcia', 'Phycisphaerae',
         'Polyangia', 'S0134.terrestrial.group', 'Subgroup.22'],
        dtype='object')},
 'Family': {'no_tuber_scab': Index(['Koribacteraceae', 'LWQ8', 'Lachnospiraceae', 'Microbacteriaceae',
         'Micrococcaceae', 'Nitrospiraceae', 'Planococcaceae', 'Rhizobiaceae',
         'Rhizobiales.Incertae.Sedis', 'Rubrital

# Feature_selection with score 0

In [ ]:
list_level = ['Class', 'Family', 'Genus', 'Order', 'Phylum']
SELECTED_FEATURE = dict.fromkeys(dirs)
SELECTED_FEATURE_0 = dict.fromkeys(dirs)
for col in range(0,len(list_level)):
    level = list_level[col]
    print(level)
    SELECTED_FEATURE[level] = dict.fromkeys(dirs)
    SELECTED_FEATURE_0[level] = dict.fromkeys(dirs)
    response_list = path_list
    feature_list = []
    for response in response_list:
        feature_list.append(all_data[response][level].index.union(results_dic[level][response][results_dic[level][response].notnull()].values))
    feature_list = [item for subitem in feature_list for item in subitem]
    feature_list = np.unique(feature_list)
    print(len(feature_list))
    matrix_df = pd.DataFrame(columns = response_list, index = feature_list)
    for response in response_list:
        for feature in feature_list:
                if (feature in all_data[response][level].index) & (feature in results_dic[level][response].values):
                    matrix_df[response].loc[feature] = 3
                elif(feature in all_data[response][level].index) &(feature not in results_dic[level][response].values):
                    matrix_df[response].loc[feature] = 2##NetComi
                elif (feature not in all_data[response][level].index) &(feature in results_dic[level][response].values):
                    matrix_df[response].loc[feature] = 1##ML
                else:
                    matrix_df[response].loc[feature] = 0##NotMLnotNetcomi
            
        #print(matrix_df[response])
#         ree=re[matrix_df[response]==3]
#         print(ree)
#         SELECTED_FEATURE[response][level]= ree.index     
        SELECTED_FEATURE[level][response] = matrix_df[response][matrix_df[response]==3].index
        print(SELECTED_FEATURE[level][response].shape[0])
        SELECTED_FEATURE_0[level][response] = matrix_df[response][matrix_df[response]==0].index[0:SELECTED_FEATURE[level][response].shape[0]]
        #print(SELECTED_FEATURE_0[level][response].shape[0])
    
    matrix_df['Sum'] = 0
    for index in matrix_df.index:
        matrix_df['Sum'].loc[index] = sum(matrix_df.loc[index].values)
    
    matrix_df.sort_values(by='Sum', ascending=False, inplace=True)
    matrix_df=matrix_df.iloc[1:30,:]

In [ ]:
path_response='/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/python code local/Main Data Files/response_sklearn/'  
for file_response in os.listdir(path_response):  
    if (file_response != '.DS_Store') & (file_response != 'Icon\r'):  
        print(file_response)  
        path_r= path_response+file_response  
        os.chdir(path_r)  
        for re in os.listdir(path_r):  
            if re[0:8] == 'response':  
                response = pd.read_csv(path_response+file_response+'/'+re)  
                response.rename(columns={'Column1':'Link_ID','x1':'y_b'}, inplace=True)  
                response.drop(columns=response.columns[2], inplace=True)  
                response=response.drop(columns='Variety2')  
                path_x = '/Users/rosa/Desktop/ALLWork/Madison/Project/Soil-nn/Code/python code local/Main Data Files/normalized_data_sklearn/'  
                writer= pd.ExcelWriter(path_r+'/'+'classification_RF_FS_notImportant'+'.xlsx', engine='xlsxwriter')   
                for file_folder in os.listdir(path_x):  
                    if (file_folder[-4:] != '.csv') & (file_folder != '.DS_Store')& (file_folder != 'Icon\r'):          
                        path = path_x+file_folder  
                        os.chdir(path)  
                        file_list = []  
                        tRF=pd.DataFrame()  
                        tcluster=pd.DataFrame()  
                        k=0  
                        for file in os.listdir(path):  
                            if (file[0] != 't') & (file[-4:] == '.csv') & (file != '.DS_Store')& (file_folder != 'Icon\r'):  
                                print(file)  
                                file_list.append(file)  
                                data_temp = pd.read_csv(file) 
                                data_temp.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True) 
                                A = [item for item in SELECTED_FEATURE_0[file_folder][file_response]]
                                A.insert(0, "Link_ID")
                                data_temp = data_temp[A]
                                data_temp.rename(columns={'Unnamed: 0':'Link_ID'}, inplace=True)  
                                data=pd.merge(response,data_temp,on='Link_ID')
                                
                                data.drop(columns = 'Link_ID',inplace=True)  
                                data_train,data_val = train_test_split(data,train_size=0.8, random_state=42)  
                                output = process_data(data_train,data_val,cv)  
                                tRF[k]=pd.DataFrame(output['f1-score'].values)         
                                k=k+1  
                        tRF.to_excel(writer, sheet_name=file_folder, index=True)  
                writer.save()                      